### This python code gives the list of contaminated genes in each positive sample

#### author: Yanan Qin

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import json
import os
import os.path
import shutil
import random
import nibabel as nib
import numpy as np
import glob
import sys
import cv2
import tensorflow as tf
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import load_model
from tensorflow import Graph
from os import path

Using TensorFlow backend.
/home/yananq/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yananq/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yananq/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yananq/anaconda3/lib/python3.7/site-packages/tenso

In [11]:
def f2(element, thresh):
    '''
    input: pixel element; thresold 
    output: 1 if element value >=threshold; 0 otherwise
    '''
    return 1 if element >= thresh else 0
f2 = np.vectorize(f2)


json = pd.read_json("/media/ssd2/yananq/mace/code/baseline/probe_correspondant.json")
map_dict  = json[0:54675]
probe2gene = map_dict.to_dict()['probe2gene']
#building a matrix with probe set ids's at their corresponding location
probe_mat = np.empty((1164, 1164), dtype=np.dtype('U100'))
for index, row in map_dict.iterrows():
    probe_set = row.name
    x_loc = row['dict'][0]
    y_loc = row['dict'][1]
    for i in range(len(x_loc)):
        probe_mat[x_loc[i],y_loc[i]] = probe_set

In [92]:
for filename in os.listdir('/media/ssd2/yananq/mace/code/pos_pred/'):# for each sample predicted as positive
    pred_new  = np.load('/media/ssd2/yananq/mace/code/pos_pred/'+filename)
    pred_new = f2(pred_new, 0.5) 

    # get the name set probesets with contamination
    probe_contam = set(probe_mat[pred_new==1])
    probe_contam = [i for i in list(probe_contam) if i != '']   #remove ''

    #get the name set of contmainated gene and non-contaminated genes
    gene_contam = list(set(probe2gene[i] for i in probe_contam if str(probe2gene[i])!= 'nan'))
    gene_contam = np.array(gene_contam)
    np.savetxt('/media/ssd/yananq/mace/contam_genes/'+filename[:-8]+'.txt', gene_contam, fmt='%s')